# Classifing Reports: NER for character (CHAR) recognition.

An important aspect of each dream report is the character that appear in it. In thi notebook, we will see how to use `dreamy` to extract character appearing in each report. As always, character are defined with respect to the Hall & Van de Castle system. CHAR are in this case spelled out, and do not/should not include the dreamer themself. Please note that CHAR data used in training is not linked to any specii feature. In other words, prediction should not be interpreted in any other way other than their presence. 

In [1]:
! pip install dreamy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 9.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import dreamy

Let's start by getting some dreams. You can stard by dowloading a collection of dream-reports scraped from the DreamBank database, freely availabe from DReAMy's hugging face!

In [3]:
dream_bank = dreamy.get_HF_DreamBank(as_dataframe=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/29345 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/DReAMy-Library___parquet/DReAMy-Library--DreamBank-dreams-34a268b12519d660/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

As you can see, the file, dowloaded directlty as a pandas DataFrame, as three entires:
- dreams, the dream-reports
- series, the different collection of DreamBank
- description, a brief description of each series

In [4]:
dream_bank.sample(5)

,dreams,series,description
10636,Some thread unraveled from my clothes and beca...,norman,Norman: a child molester
19222,It is winter: fresh snow; we have a dog: white...,emma,Emma: 48 years of dreams
24360,"(03/08/98)[""Center of Attention""] I have a bl...",b2,Barb Sanders #2
15738,Being Tested I'm traveling on train tracks. I...,kenneth,Kenneth
11119,I was putting together rectangular pieces of v...,norman,Norman: a child molester


Lets now sample a small set of dreams. If you have a more powerfull machine (or you are working on Colab), you can increase the number of report. Note than the whole dataset contains ~ 29 k reports.

In [5]:
n_samples = 10
dream_sample = dream_bank.sample(n_samples).reset_index(drop=True)

dream_as_list = dream_sample["dreams"].tolist()

Lets check which models are currently availabe.

In [6]:
dreamy.get_NER_names()

{'NER via text generation of full cahractes descriptions': 'DReAMy-lib/t5-base-DreamBank-Generation-NER-Char'}


In [7]:
dreamy.get_NER_maps()

{'full-base-en': ['DReAMy-lib/t5-base-DreamBank-Generation-NER-Char', 'summarization']}


As you can see, at the moment dreamy only has one model for NER 😅. We'll have to work with that for now 😃. Lets start by setting up the needed spec.

In [8]:
classification_type = "full"
model_type          = "base-en"
device              = "cpu"
max_length          = 512
truncation          = True
device              = "cpu"


model_name, task = dreamy.ner_model_maps[
    "{}-{}".format(classification_type, model_type)
]

Much like for the emotion-generation tutorial, do not worry about the `our max_length is set to...` warning. Is just 🤗 being nice and telling us that you could use less max-tokens.

In [9]:
predictions = dreamy.get_CHAR(
    dream_as_list, 
    model_name, 
    task,
    max_length=max_length, 
    truncation=truncation, 
    device=device,
)

Your max_length is set to 512, but you input_length is only 114. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Your max_length is set to 512, but you input_length is only 106. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=53)
Your max_length is set to 512, but you input_length is only 178. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=89)
Your max_length is set to 512, but you input_length is only 30. You might consider decreasing max_length manually, e.g. summarizer('...'

Finally, lets have a look at the prediccted characters...

In [10]:
predictions

[{'summary_text': 'group joint known child; group indefinite uncertian child; individual indefinite known adult; group male known child . group joint occupational adult;'},
 {'summary_text': 'individual female known adult; individual male uncertian adult; reative adult; original form male known adult . changed form female known child; changed form male stranger adult;'},
 {'summary_text': 'individual dead male prominent adult; individual female known adult; original form indefinite uncertian adult; changed form in late 19th century .'},
 {'summary_text': 'individual male stranger adult; group joint uncertian adult; individual female known adult; original form joint occupational adult; changed form female occupational adult .'},
 {'summary_text': "'i was in einer Gesellschaft, where ein Gericht vorkam, das Kawatanken hiess' ''"},
 {'summary_text': 'individual female known adult; group indefinite uncertian child; individual male occupational adult; individual indefinite stranger adult; u